## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
tres = 'monthly'

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
dvol = dz*darea
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [5]:
filename = f'{path}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_1948-2017_{tres}.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in [0,6,12]:
    print(ii)
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))
    #dsf[f'Phi_{ii}_0-600m'] = Phi_i.isel(z_t=range(0, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 35, 1)).sum(dim='z_t'))

0
6
12


In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:          (nlat: 140, nlon: 195)
Coordinates:
    TLAT             (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG            (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
    ULAT             (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
    ULONG            (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0_0-200m     (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_0_200-600m   (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_6_0-200m     (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_6_200-600m   (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_12_0-200m    (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_12_200-600m  (nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [7]:
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_vertical_mean_1948-2017_{tres}.nc', mode='w')

## calculate LME-averaged Phi 

In [8]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask'].isel(nlat=range(a1, a2), nlon=range(b1, b2))

In [9]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]

In [10]:
boxPhi = xr.Dataset()
for ii in [0,6,12]:
    print(ii)
    phi_box1 = np.ma.zeros(len(lme))
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    #phi_box3 = phi_box1.copy()
    for box,hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        ar1 = np.ma.zeros([len(latloc)]);ar1.mask=True
        ar2 = ar1.copy()
        #ar3 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        #    ar3[bb] = dsf[f'Phi_{ii}_0-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
        #phi_box3[hh] = np.nanmean(ar3,axis=0)
    #
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme)), dims=('lme'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme)), dims=('lme'))
    #boxPhi[f'Phi_{ii}_0-600m'] = xr.DataArray(data=phi_box3, coords=dict(lme=('lme', lme)), dims=('lme'))

0
6
12


In [11]:
boxPhi.load()

<xarray.Dataset>
Dimensions:          (lme: 11)
Coordinates:
  * lme              (lme) int64 1 2 3 4 5 6 7 8 9 10 65
Data variables:
    Phi_0_0-200m     (lme) float64 0.8194 1.125 1.442 ... 0.8652 1.989 0.8148
    Phi_0_200-600m   (lme) float64 0.1348 0.2143 0.1494 ... 0.8385 0.9208 0.1524
    Phi_6_0-200m     (lme) float64 2.237 2.148 1.573 ... 2.373 1.156 1.951
    Phi_6_200-600m   (lme) float64 0.3341 0.4533 0.2595 ... 2.245 1.19 0.3642
    Phi_12_0-200m    (lme) float64 6.181 4.206 1.859 ... 6.766 0.7059 4.691
    Phi_12_200-600m  (lme) float64 0.8317 0.9649 0.4556 ... 6.027 1.63 0.8708

In [12]:
dout = f'/glade/scratch/{USER}/DPLE-results'
boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_mean_dimension_reduced_LME_1948-2017_{tres}.nc', mode='w')